In [13]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1, bias=0),     #Input-1x28x28      Output-16x28x28    RF-3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2),
            nn.Conv2d(16, 32, 3, padding=1, bias=0),    #Input-16x28x28    Output-32x28x28    RF-5
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2, 2),                         #Input-32x28x28    Output-32x14x14    RF-10
            nn.Conv2d(32, 16, 1, padding=1, bias=0)     #Input-32x14x14    Output-16x14x14    RF-10
            
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1, bias=0),    #Input-16x14x14    Output-16x14x14    RF-12
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2),
            nn.Conv2d(16, 32, 3, padding=1, bias=0),    #Input-16x14x14    Output-32x14x14    RF-14
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.MaxPool2d(2, 2),                         #Input-32x14x14    Output-32x7x7    RF-28
            nn.Conv2d(32, 16, 1, padding=1, bias=0)     #Input-32x7x7      Output-16x7x7    RF-28
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1, bias=0),    #Input-16x7x7    Output-32x7x7    RF-30
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.Conv2d(32, 58, 1, padding=1, bias=0)     #Input-32x7x7    Output-59x7x7    RF-30
        )
        
        self.avgpool = nn.AvgPool2d(7)                  #Input-59x7x7    Output-59x1x1
        self.fc = nn.Linear(58, 10)                     
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x


In [15]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

You should consider upgrading via the '/home/sahil/miniconda3/envs/deeplearning/bin/python -m pip install --upgrade pip' command.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 32, 28, 28]           4,608
              ReLU-6           [-1, 32, 28, 28]               0
       BatchNorm2d-7           [-1, 32, 28, 28]              64
           Dropout-8           [-1, 32, 28, 28]               0
         MaxPool2d-9           [-1, 32, 14, 14]               0
           Conv2d-10           [-1, 16, 16, 16]             512
           Conv2d-11           [-1, 16, 16, 16]           2,304
             ReLU-12           [-1, 1

In [16]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [17]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [18]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 20):
    
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.2716, Accuracy: 9162/10000 (91.6%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.1012, Accuracy: 9726/10000 (97.3%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0852, Accuracy: 9752/10000 (97.5%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0698, Accuracy: 9771/10000 (97.7%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0464, Accuracy: 9858/10000 (98.6%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0727, Accuracy: 9772/10000 (97.7%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0481, Accuracy: 9858/10000 (98.6%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0734, Accuracy: 9759/10000 (97.6%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0409, Accuracy: 9865/10000 (98.7%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0333, Accuracy: 